In [2]:
# Always reload modules before executing code.
%load_ext autoreload
%autoreload 2

In [3]:
import time

import numpy as np
from matplotlib import pyplot as plt
from sklearn import datasets

import ransac

In [4]:
# Set up RANSAC problem.
n_samples = 1000
n_outliers = 50

X, y, coef = datasets.make_regression(
    n_samples=n_samples, n_features=1, n_informative=1, noise=10,
    coef=True, random_state=0)

# Add outlier data
np.random.seed(0)
X[:n_outliers] = 3 + 0.5 * np.random.normal(size=(n_outliers, 1))
y[:n_outliers] = -3 + 10 * np.random.normal(size=n_outliers)
points = np.array(list(zip([x[0] for x in X], y)))

In [5]:
%%prun -D ransac.prof

# Run RANSAC.
run_time = time.time()
ransac_model = ransac.LeastSquaresModel()
my_ransac = ransac.Ransac(ransac_model, 20, 50, 5000, 100, random_seed=0)
ransac_results = my_ransac.run(points)
run_time = time.time() - run_time
print('RANSAC ran in %0.2f seconds' % run_time)

RANSAC ran in 0.05 seconds
 
*** Profile stats marshalled to file 'ransac.prof'. 


In [6]:
# Set up XRANSAC problem.
random_state = np.random.RandomState(1)
n_samples = 500
n_outliers = 30

X, y, coef = datasets.make_regression(
    n_samples=n_samples // 2, n_features=1, n_informative=1, noise=5,
    coef=True, random_state=random_state)
X2, y2, coef2 = datasets.make_regression(
    n_samples=n_samples // 2, n_features=1, n_informative=1, noise=5,
    coef=True, random_state=random_state)

X = np.concatenate((X, X2))
y = np.array([e + 200 for e in y])
y = np.concatenate((y, y2))

X[:n_outliers] = -3 + 0.5 * random_state.normal(size=(n_outliers, 1))
y[:n_outliers] = -100 + 20 * random_state.normal(size=n_outliers)
points = np.array(list(zip([x[0] for x in X], y)))

In [7]:
%%prun -D xransac.prof

# Run XRANSAC.
ransac_model = ransac.LeastSquaresModel()
my_ransac = ransac.XRansac(
    ransac_model, 2, 50, 500, 50,
    random_seed=0, residual_histogram_num_bins=200, min_prominence=5)
run_time = time.time()
ransac_results = my_ransac.run(points)
run_time = time.time() - run_time
print('XRANSAC ran in %0.2f seconds' % run_time)

XRANSAC ran in 0.12 seconds
 
*** Profile stats marshalled to file 'xransac.prof'. 


In [9]:
%%prun -D jlink.prof

# Run J-linkage on same problem.
model = ransac.LeastSquaresModel()
jlink = ransac.JLinkage(
    model, 2, 50, 500, 50, random_seed=0)
run_time = time.time()
jlink_results = jlink.run(points)
run_time = time.time() - run_time
print('J-linkage ran in %0.2f seconds' % run_time)

J-linkage ran in 3.37 seconds
 
*** Profile stats marshalled to file 'jlink3.prof'. 
